# 本章简介
本章，我们将通过对算法进行调优来构建性能良好的机器学习模型，并对模型性能进行评估：
+ 模型性能的无偏估计
+ 处理机器学习中的常见问题
+ 机器学习模型调优
+ 使用不同的性能指标评估预测模型 5

# 6.1 基于流水线的工作流
> 介绍了如何使用sklearn中的工具类pipline使得工作更加高效  

sklearn中的pipline类使得我们可以拟合出包含任意多个处理步骤的模型，并将模型用于新数据的预测 5  
## 6.1.1 加载威斯康星乳腺癌数据集
> 介绍了如何加载威斯康星乳腺癌数据集并进行预处理  

第一步：加载数据集

In [ ]:
'''
python	sklearn	Datasets	load_breast_cancer()
python	sklearn	Datasets	load_*() as_frame
python	sklearn	Datasets	load_*() frame
'''
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer(as_frame=True)
df = bc.frame
df.head()

第二步：对类标编码，此时恶性肿瘤和良性肿瘤分别被标识为类1和类0  

In [ ]:
'''
5
'''
import numpy as np
X = df.iloc[:, :-2].values
y = df.iloc[:, -1].values
np.unique(y)

第三步：我们将数据集划分为训练数据集(80%)和单独的测试数据集(20%)  

In [ ]:
'''
5
python	sklearn	Model Selection	train_test_split()
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.20, random_state=1)

## 6.1.2 在流水线中集成数据转换及评估操作
> 介绍了如何通过pipeline实现预处理、训练等一系列操作以及pipeline的工作方式  

我们需要对数据特征列做标准化处理，此外，我们还想通过第5章介绍过的主成分分析降维到二维子空间，最后我们使用逻辑斯蒂回归模型分析数据  

In [ ]:
'''
python	sklearn	Preprocessing and Normalization	StandardScaler()
python	sklearn	Matrix Decomposition	PCA()
python	sklearn	pipeline	Pipeline()
python	sklearn	Pipeline	*.fit()
python	sklearn	Pipeline	*.score()
'''
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipe_lr = Pipeline([('scl', StandardScaler()),
                   ('pca', PCA(n_components=2)),
                   ('clf', LogisticRegression(random_state=1))])
pipe_lr.fit(X_train, y_train)
print('Test Accuracy: {:.3f}'.format(pipe_lr.score(X_test, y_test)))

Pipeline对象的输入可以是元组的序列，每个元组作为流水线的一个步骤，元组的第一个值必须是字符串，作为步骤的标识，可以用来访问具体的步骤元素，第二个值则为sklearn中的转换器、评估器  
当流水线上执行fit时，每个步骤会依次执行fit和transform操作，经过转换的数据会传递给流水线下一个对象 5   
流水线的工作方式可用下图来描述  
![6-1](../syn_pic/py_machine_learning/6-1.png)
5  
# 6.2 使用k折交叉验证评估模型性能 
> 介绍了用于模型性能评估的两种技术：holdout交叉验证和k折交叉验证的实现原理和方法  

如果一个模型过于简单会面临欠拟合（高偏差）问题，而模型过于复杂则会导致过拟合（高方差）问题。为了在偏差和方差之间找到折中的方案，我们需要对模型进行评估 5  
## 6.2.1 holdout方法
> 介绍了为何使用holdout交叉验证和其实现原理

为进一步提高模型在预测未知数据上的性能，我们还要进行模型选择过程，所谓模型选择过程：就是指针对给定分类问题我们调整参数并比较以寻求最优值  
在模型选择过程中，会产生一个问题，如果我们不断重复使用相同的测试数据，它们会成为训练数据的一部分，导致模型更易于过拟合  
所以更好的方法将数据划分为三个部分：训练数据集、验证数据集和测试数据集。其中训练数据集用于不同模型的拟合，验证数据集用于模型选择的标准  
下图介绍holdout交叉验证的概念 5  
![6-2](../syn_pic/py_machine_learning/6-2.png)
holdout方法的一个缺点在于：模型性能的评估对训练数据集划分为训练及验证子集的方法是敏感的，评价的结果会随样本的不同而发生变化 5  
## 6.2.2 k折交叉验证
> 介绍了k折交叉验证、分层k折交叉验证技术的实现原理和使用方法

k折交叉验证克服了holdout方法的缺点，对数据划分方法的敏感性较低，具有更好的鲁棒性  
其方法是不重复地随机将训练数据集划分为k个，其中k-1个用于模型的而训练，剩余的1个用于测试。随后重复此过程k次，我们就得到k个模型及对模型性能的评价  
基于这些独立且不同的数据子集上得到的模型性能评价结果，我们可以计算出其平均性能  
一旦找到了满意的超参值，我们就可以在全部训练集上重新训练模型，并使用独立测试集对模型做出最终评价 5  
由于k折交叉验证使用了无重复抽样，使得每个样本点只有一次被划入训练集或测试集的机会，与holdout方法相比，这将使得模型性能的评估具有较小的方差  
### 问题：为何无重复抽样技术可以使得模型性能评估具有较小方差?
> 待定  

下图总结了k折交叉验证的相关概念，其中k=10。$E_i$是第i次迭代的性能评价指标  
![6-3](../syn_pic/py_machine_learning/6-3.png)
k折交叉验证中的k标准值为10，但如果训练数据集相对较小，有必要加大k的值，这样会有更多的数据用于模型的训练，这样利用评估结果平均值对模型泛化性能进行评价时可以得到更小的偏差  
但是由于k值得增加导致交叉验证算法运行时间延长，而且各训练块间高度相似，也会导致评价结果方差较高。所以数据集较大，则可以选择较小得k值，如k=5 5   
k折交叉验证得一个特例就是留一(LOO)交叉验证法。在LOO中，我们将数据子集划分的数量等同于样本数(k=n)，这样每次只有一个样本用于测试，主要用于数据集非常小时的验证  
分层k折交叉验证对标准k折交叉验证做了稍许改进，它可以获得偏差和方差都较低的评估结果，特别时类别比例相差较大时  
在分k折交叉验证中，类别比例在每个分块中得以保持，下面通过sklearn中的stratifiedKFold迭代器来演示  

In [ ]:
'''
python	sklearn	Model Selection	StratifiedKFold()
python	sklearn	Model Selection	*.split()
python	numpy	Statistics	bincount()
'''
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10)
scores = []
for k, (train, test) in enumerate(kfold.split(X_train, y_train)):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: {}, Class dist.：{}， Acc：{:.3f}'.format(k+1, 
                                                        np.bincount(y_train[train]), score))

In [ ]:
'''
5
python	numpy	Statistics	mean()
python	numpy	Statistics	std()
'''
print('CV accuracy: {:.3f} +/- {:.3f}'.format(np.mean(scores),
                                             np.std(scores)))

我们test索引计算模型的准确率，将其存储在score列表中，用于计算平均准确率及性能评估标准差  
另外sklearn也同样实现了k折交叉验证评分的计算  

In [ ]:
'''
python	sklearn	Model Selection	cross_val_score()
'''
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=pipe_lr,
                        X=X_train,
                        y=y_train,
                        cv=10,
                        n_jobs=1)
print('CV accuracy scores:{}'.format(scores))

In [ ]:
print('CV accuracy: {:.3f} +/- {:.3f}'.format(np.mean(scores),
                                             np.std(scores)))

cross_val_score通过设定n_jobs可以支持多CPU并行运算 5  
# 6.3 通过学习及验证曲线来调试算法
> 本章介绍了如何使用两个有助于提高学习算法性能的判定工具：学习曲线与验证曲线 

学习曲线和验证曲线可以用来判定学习算法是否面临过拟合（高方差）或欠拟合（高偏差）问题 5  
## 6.3.1 使用学习曲线判定偏差和方差问题  
> 介绍了如何运用学习曲线来判定模型是否拟合良好，以及是否应该收集更多数据  

通过将模型的训练及准确性验证看作是训练数据集大小的函数，并绘制其图像，我们可以很容易看出模型是面临高方差还是高偏差的问题  
![6-4](../syn_pic/py_machine_learning/6-4.png)
左上方显示的是一个高偏差模型，解决此问题的常用方法是增加模型中参数的数量，例如，收集或构建额外特征，或者降低类似于SVM等模型的正则化程度  
右上方图像中的模型面临高方差的问题，表明训练准确度与交叉验证准确度之间有很大差距 5  
针对此类过拟合问题，我们可以收集更多的训练数据或者降低模型的复杂度，如正则化参数。或用特征抽取来降低特征的数量  
需要注意：收集更多数据不适用于所有问题，例如训练数据中噪声过多，或者模型本身接近最优  

In [ ]:
'''
5
python	sklearn	Pipeline	Pipeline()
python	sklearn	Model Selection	learning_curve()
python	matplotlib	Pyplot function overview	fill_between()
python	matplotlib	Pyplot function overview	grid()
b
'''
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
pipe_lr = Pipeline([
    ('scl', StandardScaler()),
    ('clf', LogisticRegression(
        penalty='l2', random_state=0))])
train_sizes, train_scores, test_scores =\
    learning_curve(estimator=pipe_lr,
                  X=X_train,
                  y=y_train,
                  train_sizes=np.linspace(0.1, 1.0, 10),
                  cv=10,
                  n_jobs=1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.plot(train_sizes, train_mean,
        color='blue', marker='o',
        markersize=5,
        label='training accuracy')
plt.fill_between(train_sizes,
                train_mean + train_std,
                train_mean - train_std,
                alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean,
        color='green', linestyle='--',
        marker='s',markersize=5,
        label='validation accuracy')
plt.fill_between(train_sizes,
                test_mean + test_std,
                test_mean - test_std,
                alpha=0.15, color='green')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.show()

通过learning_curve函数的train_sizes参数，我们可以控制用于生成学习曲线的样本的决定或相对数量。learning_curve函数使用分层K折交叉验证来计算交叉验证准确率  
我们通过fill_between函数加入了平均准确率标准差的信息，用以标识评价结果的标准差  
由于训练准确率曲线稍高于验证准确率，意味着模型对训练数据有轻微的过拟合，可以通过增大正则化来解决 5  
## 6.3.2 通过验证曲线来判定过拟合与欠拟合  
> 介绍了如何运用验证曲线来判定模型是否拟合良好，以及是否需要调整模型参数  

验证曲线与学习曲线相似，不过绘制的不是样本大小与训练准确率、测试准确率之间的函数关系，而是准确率与模型参数之间的关系  

In [ ]:
'''
python	sklearn	Model Selection	validation_curve()
'''
from sklearn.model_selection import validation_curve
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
train_scores, test_scores =\
    validation_curve(estimator=pipe_lr,
                  X=X_train,
                  y=y_train,
                  param_name='clf__C',
                  param_range=param_range,
                  cv=10)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.plot(param_range, train_mean,
        color='blue', marker='o',
        markersize=5,
        label='training accuracy')
plt.fill_between(param_range,
                train_mean + train_std,
                train_mean - train_std,
                alpha=0.15, color='blue')
plt.plot(param_range, test_mean,
        color='green', linestyle='--',
        marker='s',markersize=5,
        label='validation accuracy')
plt.fill_between(param_range,
                test_mean + test_std,
                test_mean - test_std,
                alpha=0.15, color='green')
plt.grid()
plt.xscale('log')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.show()

在validation_curve函数内，我们可以指定想要验证的参数。例如，需要验证参数C，即定义在流水线中逻辑斯蒂回归分类器的正则化参数，我们将其记为'clf_C'  
我们可以看到，如果加大正则化强度（较小的C值），会导致模型轻微的欠拟合;如果增加C的值，这意味着降低正则化强度，模型会趋向于过拟合，最优点在C=0.1附近 5  
# 6.4 使用网格搜索调优机器学习模型
> 介绍了调优模型超参的技术-网格搜索，以及在不同算法中做出选择的技术-嵌套交叉验证的概念和使用方法  

机器学习中的两类参数：通过训练数据学习得到的参数，以及学习算法中需要单独进行优化的参数  
后者称为调优参数，也称为超参，如LR中的正则化系数或者决策树的深度参数 5  
## 6.4.1 使用网格搜索调优超参
> 介绍了网格搜索使用方法

网格搜索法的原理是通过我指定的不同超参列表进行暴力穷举搜索，并计算评估每个组合对模型性能的影响  

In [ ]:
'''
python	sklearn	Model Selection	GridSearchCV()
python	sklearn	Model Selection	*.best_score_
'''
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
pipe_svc = Pipeline([('scl', StandardScaler()),
                    ('clf', SVC(random_state=1))])
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'clf__C': param_range,
              'clf__kernel': ['linear']},
             {'clf__C': param_range,
              'clf__gamma': param_range,
              'clf__kernel': ['rbf']}]
gs = GridSearchCV(estimator=pipe_svc,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)

In [ ]:
'''
python	sklearn	Model Selection	*.best_params_
'''
print(gs.best_params_)